# HPE Container Platform (HPECP) Automate Splunk Application Deployments with HPECP  - Lab 1
## Deploying cloud native and non-cloud native applications, programmatically on Kubernetes clusters managed by the HPE Container Platform.


**Requirements:**
- This workshop builds off of HackShack W479 but is not required to use this workshop
- HPE Container Platform deployment with a managed Kubernetes cluster
- IP address or FQDN of the HPE Container Platform's controller host and Gateway host
- a Kubernetes tenant user account   

**Utilities:**   
- cURL  
- Jupyter Notebook server with bash kernel installed
- kubectl


**Lab workflow:**

In this lab we will cover:

1. As tenant user, you will first establish a valid login session with HPECP using the HPECP REST API.

2. You will then fetch the Kubeconfig file for your tenant working context using the HPECP REST API.
>Note: A kubeconfig file is used to access the Kubernetes cluster for your tenant working context

3. Deploy the Splunk Operator for Kubernetes which leverages as well as a Splunk stand alone application.
>Note: A kubeconfig file is used to access the Kubernetes cluster for your tenant working context

4. Interact with your Splunk application using cURL.

**Definitions:**

- *Cloud native application:* Also known as the 12 Factor app, a modern application that leverages microservices architecture with loosely coupled services.

- *Non-cloud native application:* a multi-tier application with tightly coupled and interdependent services. 

- *Stateless application:* A stateless application is an application which does not require persistence of data nor an application state.

- *Stateful application:* A stateful application typically requires persistence of certain mountpoints across application cluster nodes rescheduling, restarts, upgrades, rollbacks. A stateful application can also need persistence of network identity (i.e.: hostname). 

- *Splunk Operator for Kubernets (SOK)* The Splunk Operator for Kubernetes (SOK) makes it easy for Splunk Administrators to deploy and operate Enterprise deployments in a Kubernetes infrastructure. Packaged as a container, it uses the operator pattern to manage Splunk-specific custom resources, following best practices to manage all the underlying Kubernetes objects for you.

#### -1- Establish A Login Session With The HPE CP Enviroment
#### Initialize the environment:

Let's first define the environment variables according to your HPE Container Platform user account and tenant name, and the HPE Container Platform API system endpoint:

In [ ]:
#
# environment variables to be adjusted/verified by the student
#

student="student$$I" # your Jupyter Notebook student Identifier (i.e.: student<xx>)
username="student$$I" # your HPECP tenant login credentials - username 
password="$$PASSSTU" # your HPECP tenant login credentials - password
#
# fixed environment variables setup by the HPECP lab administrator
#
controller_endpoint="gateway1.hpedevlab.net:8080"
gateway_host="gateway1.hpedevlab.net"
tenantname="K8sHackTenant"
k8sClusterId="1"  #this is the K8s Cluster Id provided by the HPECP admisnistrator and assigned to your K8s tenant.

echo "your working context is:" $username-$password-$tenantname 

#### Authenticate as a Tenant user in the specified tenant:

In [ ]:
sessionlocation=$(curl -k -i -s --request POST "https://${controller_endpoint}/api/v2/session" \
--header 'Accept: application/json' \
--header 'Content-Type: application/json' \
--data-raw '{
"name": "'"$username"'",
"password": "'"$password"'",
"tenant_name": "'"$tenantname"'"
}' | grep Location | awk '{print $2}' | tr -d '\r') #we remove any cr that might exist
echo "This is your session location: " $sessionlocation
SessionId=$(echo $sessionlocation | cut -d'/' -f 5) # extract sessionId for later, for logout
echo "This is your session_Id:" $SessionId

#### Quickly check to ensure you can make REST API calls within your tenant working context:
Here you will fetch information about the session you have just established.

In [ ]:
curl -k -s --request GET "https://${controller_endpoint}/api/v2/session/${SessionId}" \
--header "X-BDS-SESSION: $sessionlocation" \
--header 'Accept: application/json' \
--header 'Content-Type: application/json' | jq  #using jq to pretty print the JSON reponse of the API call 

#### Get the Kubeconfig file for your tenant working context:
The next step in deploying a containerized application in Kubernetes clusters managed by the HPE Container Platform is to get the kubeconfig file for your tenant working context. 

The HPECP REST API call below allows you to get the **kubeconfig file** used to access the Kubernetes cluster for your tenant user account based on your assigned role (tenant member) with the same result as if you had downloaded it from the HPECP UI.

In [ ]:
curl -k -s --request GET "https://${controller_endpoint}/api/v2/k8skubeconfig" \
--header "X-BDS-SESSION: $sessionlocation" \
--header 'Accept: application/json' \
--header 'Content-Type: application/json' > ${student}_kubeconfig

In [ ]:
cat ${student}_kubeconfig

> Notice the kubeconfig file includes the valid token (session location) for the current session you previously established.

#### Define the Kubeconfig file as a shell environment variable:
The kubectl command-line tool (a K8s API client) uses kubeconfig file to communicate with the Kube API server of a Kubernetes cluster. By default, kubectl looks for a file named **config** in the $HOME/.kube directory. Because our kubeconfig file is not located in a default location, we must specify the path of the kubeconfig file by setting the KUBECONFIG environment variable:

In [ ]:
#define the Kubeconfig file as a shell environment variable to tell kubectl where to look for the kubeconfig file
export KUBECONFIG="${student}_kubeconfig"
echo $KUBECONFIG

#### Check your working tenant context:

In [ ]:
kubectl config view

You can now send Kubernetes API requests using a K8s API client such as **kubectl** to deploy enterprise workloads to the kubernetes cluster using the privileges assigned to your tenant role. 

>Note: kubectl has been installed in the JupyterHub server by the IT administrator.

**Let's see this in action!**

## -2- Setup and install Splunk Operator for Kubernets (SOK) within your pod.

#### First let's ensure we can talk to our k8s resources by running a kubectl command. We will likely see other pods running from other users on this namespace, but that's ok:

In [ ]:
kubectl get pod

#### Intall the Splunk Operator
Note*:  When you see a "Asterisk" symbol next to the next action it means your execution step is busy working within the notebook.

In [ ]:
kubectl apply -f http://tiny.cc/splunk-operator-install

In [ ]:
kubectl get pod

We should see our Spark Operator running.  It will look like: splunk-operator-(random)-(random)

## Launch the Splunk Stand Alone Application

First we need to create the YAML file for Splunk stand alone

In [ ]:
cat > 0_splunkStandAlone.yaml << EOF
apiVersion: enterprise.splunk.com/v1alpha2
kind: Standalone
metadata:
  name: $username
  finalizers:
  - enterprise.splunk.com/delete-pvc
spec:
  storageClassName: "openebs-hostpath"
EOF

In [ ]:
kubectl apply -f 0_splunkStandAlone.yaml

In [ ]:
kubectl get pod

Above you will see your application was deployed as 'splunk-user_student_id-standealone-number-increment'

Expose your Splunk applications port

In [ ]:
cat > mySplunkService.yaml << EOF
apiVersion: v1
kind: Service
metadata:
  name: splunk-$username-standalone-0
  namespace: k8shacktenant
spec:
  ports:
  - name: http-splunk 
    port: 8000
    protocol: TCP
    targetPort: 8000
  selector:
    app.kubernetes.io/component: standalone
    app.kubernetes.io/instance: splunk-$username-standalone
    app.kubernetes.io/managed-by: splunk-operator
    app.kubernetes.io/name: standalone
    app.kubernetes.io/part-of: splunk-$username-standalone
    statefulset.kubernetes.io/pod-name: splunk-$username-standalone-0
  sessionAffinity: None
  type: NodePort
status:
  loadBalancer: {}

EOF

In [ ]:
kubectl apply -f mySplunkService.yaml

## -3- Get the Kubernetes Pod and Service for your deployed application
HPE Container Platform automatically maps the **NodePort Service** endpoint to the HPE Container Platform gateway (proxy) host.
Access to application services running in containers is proxied via the gateway host on a publicly-accessible IP address and a port number greater than 10000.

In [ ]:
myservice="splunk-${username}-standalone-0"
appURL=$(kubectl describe service/"${myservice}" | grep gateway1 | awk '{print $3}')
appPort=$(echo $appURL | cut -d':' -f 2) # extract the gateway re-mapped port value.
myapp_endpoint="https://$gateway_host:$appPort"
echo "Your application service endpoint re-mapped port is: "$appPort
echo "Your Intranet application service endpoint is: "$myapp_endpoint

## -4- Check to see if your application is responding and log in:

In order to login to your deployed Splunk application you will need the password:

In [ ]:
splunkPW=$(kubectl get secret splunk-${username}-standalone-secrets -o jsonpath='{.data.password}' | base64 --decode)
echo $splunkPW

**If the below command fails, wait approximately one minute as the service comes up**

In [ ]:
curl -k ${myapp_endpoint}

>Note:  If you see a "No server is available to handle this request" retry the previous step a couple of times.

>Note:  If you see a message that says "...The resource has moved temporarily" your applicaiton is running correctly.  

Alternatively: You can also connect to your application from your browser through our Internet NAT firewall:   
Open a new tab in your browser and connect to the service endpoint: https://notebooks.hpedev.io:$appPort where the port number is the re-mapped port you got above for your service endpoint.  (Please note that some browswers have issues with the SSL self signed certificates, try another broweser if you are having issues)

Do a copy (CTRL-C) of the URL after execution of the cell code below and paste it (CTRL-V) on a new tab in your browser.

In [ ]:
echo "Your application service is also accessible from your browser at:" "https://notebooks.hpedev.io:$appPort"
echo User Login: admin
echo Password: $splunkPW

## -5- Delete your stateless application deployment:
Delete your application deployment and services using the K8s API request below. The K8s API call requires the kubectl operation type (delete) and the application YAML manifest.
After a minute or so, you should get the message: deployment deleted and service deleted.

In [ ]:
kubectl delete standalone $username
kubectl delete -f mySplunkService.yaml

In [ ]:
curl -k -i -s --request DELETE "https://${controller_endpoint}/api/v2/session/${SessionId}" \
--header "X-BDS-SESSION: $sessionlocation" \
--header 'Accept: application/json' \
--header 'Content-Type: application/json'

The status *204 No Content* means the session object has been deleted.

* Finally, reset your applications YAML files and delete the kubeconfig file

In [ ]:
#reset the application deployment name in the YAML file and delete the kubeconfgi file
rm ${student}_kubeconfig
rm mySplunkService.yaml
rm 0_splunkStandAlone.yaml

## Summary

In this tutorial, we have shown you how to deploy the Splunk Operator for Kubernetes on the HPE Container Platform. More information regarding the SOK can be found here: https://github.com/splunk/splunk-operator/blob/master/docs/README.md.

Now, let's look at how to look at a variety of data source generated by the HPECP

* [Understanding the Data Provided by the HPECP](2-WKSHP-HPECP-Charge-back-and-reporting.ipynb)